In [1]:
import requests
import folium
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

base_url = 'http://openapi.seoul.go.kr:8088/5371655259646f6f3335576e4f4474/json/bikeList/1/5/'
response = requests.get(base_url)
# print(request)
json_data = response.json()
json_data

{'rentBikeStatus': {'list_total_count': 5,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'},
  'row': [{'rackTotCnt': '15',
    'stationName': '102. 망원역 1번출구 앞',
    'parkingBikeTotCnt': '5',
    'shared': '33',
    'stationLatitude': '37.55564880',
    'stationLongitude': '126.91062927',
    'stationId': 'ST-4'},
   {'rackTotCnt': '14',
    'stationName': '103. 망원역 2번출구 앞',
    'parkingBikeTotCnt': '10',
    'shared': '71',
    'stationLatitude': '37.55495071',
    'stationLongitude': '126.91083527',
    'stationId': 'ST-5'},
   {'rackTotCnt': '13',
    'stationName': '104. 합정역 1번출구 앞',
    'parkingBikeTotCnt': '8',
    'shared': '62',
    'stationLatitude': '37.55073929',
    'stationLongitude': '126.91508484',
    'stationId': 'ST-6'},
   {'rackTotCnt': '5',
    'stationName': '105. 합정역 5번출구 앞',
    'parkingBikeTotCnt': '1',
    'shared': '20',
    'stationLatitude': '37.55000687',
    'stationLongitude': '126.91482544',
    'stationId': 'ST-7'},
   {'rackTotCnt': '12',
  

In [2]:
json_data.get("rentBikeStatus", {}).get("RESULT", {}).get("CODE", "")

'INFO-000'

In [3]:
def fetch_bike_data():
    base_url = "http://openapi.seoul.go.kr:8088/5371655259646f6f3335576e4f4474/json/bikeList/"
    start = 1
    end = 1000
    step = 1000
    data_frames = []
    while True:
        # "http://openapi.seoul.go.kr:8088/4a4e64575972797537364363425073/json/bikeList/1/1000"
        url = f'{base_url}{start}/{end}/'
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Status Code: {response.status_code}")
            break
        json_data = response.json()
        try:
            rent_bike_status = json_data['rentBikeStatus']
            result_code = rent_bike_status['RESULT']['CODE']
        except KeyError:
            print('JSON 오류')
            break
        if result_code == "INFO-200":
            print('데이터 없음')
            break
        elif result_code == "INFO-000":
            print(f'시작: {start} 끝: {end}')
            try:
                bike_data = rent_bike_status['row']
                if bike_data:
                    df = pd.DataFrame(bike_data)
                    data_frames.append(df)
            except KeyError:
                print('데이터를 찾을 수 없음')
        elif result_code == "ERROR-336":
            print('데이터요청은 한번에 최대 1,000건을 넘을 수 없습니다')
            break
        else:
            print(f'result code: {result_code}')
            break
        start += step
        end += step
    if data_frames:
        final_df = pd.concat(data_frames, ignore_index=True)
        return final_df
    else:
        return pd.DataFrame()

In [4]:
bike_data_df = fetch_bike_data()

시작: 1 끝: 1000
시작: 1001 끝: 2000
시작: 2001 끝: 3000
JSON 오류


In [5]:
bike_data_df

,rackTotCnt,stationName,parkingBikeTotCnt,shared,stationLatitude,stationLongitude,stationId
0,15,102. 망원역 1번출구 앞,5,33,37.55564880,126.91062927,ST-4
1,14,103. 망원역 2번출구 앞,10,71,37.55495071,126.91083527,ST-5
2,13,104. 합정역 1번출구 앞,8,62,37.55073929,126.91508484,ST-6
3,5,105. 합정역 5번출구 앞,1,20,37.55000687,126.91482544,ST-7
4,12,106. 합정역 7번출구 앞,9,75,37.54864502,126.91282654,ST-8
...,...,...,...,...,...,...,...
2753,8,6185.가양나들목,6,75,37.57341003,126.84345245,ST-3418
2754,12,6187.마곡119안전센터 맞은편,18,150,37.55534744,126.82072449,ST-3415
2755,10,6188.금호아파트,14,140,37.55619049,126.86463928,ST-3419
2756,11,6189.데시앙플렉스 지식산업센터,5,45,37.56448364,126.84830475,ST-3424


In [6]:
bike_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2758 entries, 0 to 2757
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   rackTotCnt         2758 non-null   object
 1   stationName        2758 non-null   object
 2   parkingBikeTotCnt  2758 non-null   object
 3   shared             2758 non-null   object
 4   stationLatitude    2758 non-null   object
 5   stationLongitude   2758 non-null   object
 6   stationId          2758 non-null   object
dtypes: object(7)
memory usage: 151.0+ KB


In [7]:
bike_data_df.columns

Index(['rackTotCnt', 'stationName', 'parkingBikeTotCnt', 'shared',
       'stationLatitude', 'stationLongitude', 'stationId'],
      dtype='object')

In [8]:
bike_data_df.shape

(2758, 7)

In [9]:
bike_data_df['stationLongitude'] = bike_data_df['stationLongitude'].astype(float)
bike_data_df['stationLatitude'] = bike_data_df['stationLatitude'].astype(float)

In [10]:
bike_map = folium.Map(location=[bike_data_df['stationLatitude'].mean(),
                                bike_data_df['stationLongitude'].mean()],
                                zoom_start=12)

for index, data in bike_data_df.iterrows():
    popup_str = '{} 자전거주차총건수:{}대'.format(
        data['stationName'], data['parkingBikeTotCnt']
    )
    popup = folium.Popup(popup_str, max_width=600)
    folium.Marker(location=[data['stationLatitude'], data['stationLongitude']],
                  popup=popup).add_to(bike_map)

bike_map